# Análise dos dados da COVID-19 no ES
## Maiky Barreto da Silva

Pontos a serem abordados:
- Evolução da pandemia no Estado, com dados diários, semanais e médias móveis
- Evolução da pandemia na GV (Vitória, Vila Velha, Cariacica, Serra, Viana)
- Motivos que levam as pessoas a irem aos postos de saúde

In [61]:
# Pacotes básicos para a análise

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import seaborn
seaborn.set()

In [62]:
# Pacotes de apresentação de dados

import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

In [63]:
# Leitura dos dados

df = pd.read_csv('https://bi.static.es.gov.br/covid19/MICRODADOS.csv', sep = ';', encoding = 'iso8859_15')

In [64]:
df.head()

,DataNotificacao,DataCadastro,DataDiagnostico,DataColeta_RT_PCR,DataColetaTesteRapido,DataEncerramento,DataObito,Classificacao,Evolucao,CriterioConfirmacao,...,ComorbidadeRenal,ComorbidadeDiabetes,ComorbidadeTabagismo,ComorbidadeObesidade,FicouInternado,ViagemBrasil,ViagemInternacional,ProfissionalSaude,PossuiDeficiencia,MoradorDeRua
0,2020-09-07,2020-09-07,2020-09-05,NaN,NaN,NaN,NaN,Suspeito,-,-,...,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não
1,2020-09-07,2020-09-07,2020-09-03,2020-09-07,NaN,NaN,NaN,Suspeito,-,-,...,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não
2,2020-09-07,2020-09-07,2020-09-02,2020-09-07,NaN,NaN,NaN,Suspeito,-,-,...,Não,Não,Não,Não,Não Informado,Não,Não Informado,Não,Não,Não
3,2020-09-07,2020-09-07,2020-09-03,NaN,NaN,NaN,NaN,Suspeito,-,-,...,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não
4,2020-09-07,2020-09-07,2020-07-01,NaN,2020-07-20,2020-09-07,NaN,Descartados,Ignorado,Laboratorial,...,Não,Não,Não,Não,Não Informado,Não Informado,Não Informado,Sim,Não,Não


In [65]:
df.shape

(352436, 37)

In [66]:
df.DataNotificacao = df.DataNotificacao.astype(np.datetime64)
df.DataObito = df.DataObito.astype(np.datetime64)

Primeiro serão obtidos os gráficos de barras diário, semanal e média móvel de casos no ES. Idem para os óbitos.

In [67]:
df_conf = df.query("Classificacao == 'Confirmados'")
df_conf.head()

,DataNotificacao,DataCadastro,DataDiagnostico,DataColeta_RT_PCR,DataColetaTesteRapido,DataEncerramento,DataObito,Classificacao,Evolucao,CriterioConfirmacao,...,ComorbidadeRenal,ComorbidadeDiabetes,ComorbidadeTabagismo,ComorbidadeObesidade,FicouInternado,ViagemBrasil,ViagemInternacional,ProfissionalSaude,PossuiDeficiencia,MoradorDeRua
22,2020-09-07,2020-09-07,2020-06-21,2020-09-06,2020-09-06,NaN,NaT,Confirmados,-,Laboratorial,...,Não,Sim,Não,Não,Sim,Não,Não,Não,Não,Não
30,2020-09-07,2020-09-07,2020-08-28,2020-09-06,2020-09-05,NaN,NaT,Confirmados,-,Laboratorial,...,Não,Sim,Não,Não,Sim,Não,Não,Não,Não,Não
32,2020-09-07,2020-09-07,2020-08-28,2020-09-05,2020-09-05,NaN,NaT,Confirmados,-,Laboratorial,...,Não,Não,Não,Não,Sim,Não,Não,Não,Não,Não
51,2020-09-07,2020-09-07,2020-09-03,2020-09-07,2020-09-06,NaN,NaT,Confirmados,-,Laboratorial,...,Não,Sim,Não,Não,Não Informado,Não,Não,Não,Não,Não
71,2020-09-07,2020-09-07,2020-08-30,NaN,2020-09-07,NaN,NaT,Confirmados,-,Clinico Epdemiologico,...,Não,Sim,Não,Não,Não,Não,Não,Não,Não,Não


In [68]:
total_notif_ES = df_conf.groupby('DataNotificacao').count().DataCadastro
total_notif_ES

DataNotificacao
2020-02-29      1
2020-03-06      1
2020-03-13      4
2020-03-14      2
2020-03-15      3
             ... 
2020-09-03    472
2020-09-04    376
2020-09-05     63
2020-09-06     21
2020-09-07     18
Name: DataCadastro, Length: 181, dtype: int64

In [69]:
rm_notif_ES = total_notif_ES.rolling(14).mean()
rm_notif_ES

DataNotificacao
2020-02-29           NaN
2020-03-06           NaN
2020-03-13           NaN
2020-03-14           NaN
2020-03-15           NaN
                 ...    
2020-09-03    461.500000
2020-09-04    441.071429
2020-09-05    427.500000
2020-09-06    417.357143
2020-09-07    363.071429
Name: DataCadastro, Length: 181, dtype: float64

In [70]:
def figura_midia(regiao, tipo, total, rm):
    if tipo == 'mortes':
        titulo = 'Mortes por COVID-19 -'+regiao+'- Diário'
        legenda = 'Óbitos diários'
        yaxes = 'Pacientes mortos'
    elif tipo == 'casos':
        titulo = 'Casos confirmados de COVID-19 - '+regiao+' - Diário'
        legenda = 'Notificações diárias'
        yaxes = 'Pacientes infectados'
    else:
        return print('Tipo inválido')
    
    fig = go.Figure()
    fig.add_bar(x = total.index, y = total, name = legenda)
    fig.add_scatter(x = rm.index, y = rm, name = 'Média móvel 14d')
    fig.update_layout(title = titulo)
    fig.update_xaxes(title = 'Data de Notificação')
    fig.update_yaxes(title = 'Pacientes infectados')
    fig.update_traces(hovertemplate = 'Data: %{x}<br>Casos no dia: %{y}', selector={'name':legenda})
    fig.update_traces(hovertemplate = 'Data: %{x}<br>Média móvel: %{y}', selector={'name':'Média móvel 14d'})
    fig.update_layout(xaxis=dict(
            rangeselector=dict(
                buttons=list([
                    dict(count=1,
                        label="1m",
                        step="month",
                        stepmode="backward"),
                        dict(count=3,
                        label="3m",
                        step="month",
                        stepmode="backward"),
                        dict(step="all")
                        ])),
            rangeslider=dict(
                visible=True
            ),type="date"))
    fig.show()

In [71]:
figura_midia('ES', 'casos', total_notif_ES, rm_notif_ES)

In [72]:
px.bar(total_notif_ES.resample('W').sum(), title = 'Casos confirmados no ES - Semanal')

In [73]:
total_ob_ES = df_conf.groupby('DataObito').count().DataCadastro
total_ob_ES

DataObito
2020-03-13     1
2020-04-01     4
2020-04-02     3
2020-04-04     1
2020-04-10     2
              ..
2020-09-03    15
2020-09-04    14
2020-09-05    12
2020-09-06     8
2020-09-07     4
Name: DataCadastro, Length: 155, dtype: int64

In [74]:
rm_ob_ES = total_ob_ES.rolling(14).mean()
rm_ob_ES

DataObito
2020-03-13          NaN
2020-04-01          NaN
2020-04-02          NaN
2020-04-04          NaN
2020-04-10          NaN
                ...    
2020-09-03    16.142857
2020-09-04    16.142857
2020-09-05    15.785714
2020-09-06    15.000000
2020-09-07    13.928571
Name: DataCadastro, Length: 155, dtype: float64

In [75]:
figura_midia('ES', 'mortes', total_ob_ES, rm_ob_ES)

In [76]:
px.bar(total_ob_ES.resample('W').sum(), title = 'Mortes por COVID-19 no ES - Semanal')

Isolando os dados a GV:

In [77]:
gv = ['VILA VELHA', 'VITÓRIA', 'CARIACICA', 'VIANA', 'SERRA']
df_conf_GV = df_conf.query("Municipio == @gv")
df_conf_GV.head()

,DataNotificacao,DataCadastro,DataDiagnostico,DataColeta_RT_PCR,DataColetaTesteRapido,DataEncerramento,DataObito,Classificacao,Evolucao,CriterioConfirmacao,...,ComorbidadeRenal,ComorbidadeDiabetes,ComorbidadeTabagismo,ComorbidadeObesidade,FicouInternado,ViagemBrasil,ViagemInternacional,ProfissionalSaude,PossuiDeficiencia,MoradorDeRua
22,2020-09-07,2020-09-07,2020-06-21,2020-09-06,2020-09-06,NaN,NaT,Confirmados,-,Laboratorial,...,Não,Sim,Não,Não,Sim,Não,Não,Não,Não,Não
30,2020-09-07,2020-09-07,2020-08-28,2020-09-06,2020-09-05,NaN,NaT,Confirmados,-,Laboratorial,...,Não,Sim,Não,Não,Sim,Não,Não,Não,Não,Não
198,2020-09-07,2020-09-07,2020-08-26,2020-09-06,2020-09-06,NaN,NaT,Confirmados,-,Laboratorial,...,Não,Sim,Não,Não,Sim,Não,Não,Não,Não,Não
208,2020-09-07,2020-09-07,2020-09-02,2020-09-07,2020-09-07,NaN,NaT,Confirmados,-,Laboratorial,...,Não,Não,Não,Sim,Sim,Não,Não,Não,Não,Não
240,2020-09-07,2020-09-07,2020-07-15,2020-09-06,2020-09-06,NaN,NaT,Confirmados,-,Laboratorial,...,Não,Sim,Não,Não,Sim,Não,Não,Não,Não,Não


In [78]:
total_notif_GV = df_conf_GV.groupby('DataNotificacao').count().DataCadastro
total_notif_GV

DataNotificacao
2020-02-29     1
2020-03-06     1
2020-03-13     2
2020-03-14     1
2020-03-15     2
              ..
2020-09-03    96
2020-09-04    94
2020-09-05    25
2020-09-06     9
2020-09-07     9
Name: DataCadastro, Length: 181, dtype: int64

In [79]:
rm_notif_GV = total_notif_GV.rolling(14).mean()
rm_notif_GV

DataNotificacao
2020-02-29           NaN
2020-03-06           NaN
2020-03-13           NaN
2020-03-14           NaN
2020-03-15           NaN
                 ...    
2020-09-03    109.000000
2020-09-04    103.928571
2020-09-05     98.214286
2020-09-06     95.357143
2020-09-07     85.857143
Name: DataCadastro, Length: 181, dtype: float64

In [80]:
figura_midia('GV', 'casos', total_notif_GV, rm_notif_GV)

In [81]:
px.bar(total_notif_GV.resample('W').sum(), title = 'Casos confirmados na GV - Semanal')

In [82]:
total_ob_GV = df_conf_GV.groupby('DataObito').count().DataCadastro
total_ob_GV

DataObito
2020-03-13    1
2020-04-01    2
2020-04-02    2
2020-04-10    2
2020-04-11    1
             ..
2020-09-03    2
2020-09-04    5
2020-09-05    4
2020-09-06    3
2020-09-07    1
Name: DataCadastro, Length: 154, dtype: int64

In [83]:
rm_ob_GV = total_ob_GV.rolling(14).mean()
rm_ob_GV

DataObito
2020-03-13         NaN
2020-04-01         NaN
2020-04-02         NaN
2020-04-10         NaN
2020-04-11         NaN
                ...   
2020-09-03    4.357143
2020-09-04    4.428571
2020-09-05    4.142857
2020-09-06    3.928571
2020-09-07    3.357143
Name: DataCadastro, Length: 154, dtype: float64

In [84]:
figura_midia('GV', 'mortes', total_ob_GV, rm_ob_GV)

In [85]:
px.bar(total_ob_GV.resample('W').sum(), title = 'Óbitos na GV - Semanal')

In [86]:
# Limpeza de memória

del rm_notif_ES, rm_notif_GV, total_notif_ES, total_notif_GV, total_ob_ES, total_ob_GV

Verificando as características das pessoas que procuraram os postos de saúde:

In [87]:
df.columns

Index(['DataNotificacao', 'DataCadastro', 'DataDiagnostico',
       'DataColeta_RT_PCR', 'DataColetaTesteRapido', 'DataEncerramento',
       'DataObito', 'Classificacao', 'Evolucao', 'CriterioConfirmacao',
       'StatusNotificacao', 'Municipio', 'Bairro', 'FaixaEtaria',
       'IdadeNaDataNotificacao', 'Sexo', 'RacaCor', 'Escolaridade', 'Febre',
       'DificuldadeRespiratoria', 'Tosse', 'Coriza', 'DorGarganta', 'Diarreia',
       'Cefaleia', 'ComorbidadePulmao', 'ComorbidadeCardio',
       'ComorbidadeRenal', 'ComorbidadeDiabetes', 'ComorbidadeTabagismo',
       'ComorbidadeObesidade', 'FicouInternado', 'ViagemBrasil',
       'ViagemInternacional', 'ProfissionalSaude', 'PossuiDeficiencia',
       'MoradorDeRua'],
      dtype='object')

In [88]:
df.shape[0]

352436

In [89]:
sintomas = ['Febre','DificuldadeRespiratoria', 'Tosse', 'Coriza', 'DorGarganta', 'Diarreia','Cefaleia']
motivos = df.groupby(sintomas).count().DataNotificacao/df.shape[0]*100
motivos.head()

Febre  DificuldadeRespiratoria  Tosse  Coriza  DorGarganta  Diarreia  Cefaleia
-      -                        -      -       -            -         -           0.362052
                                                                      Não         0.001419
                                                                      Sim         0.000284
                                                            Não       -           0.001419
                                               Não          -         -           0.001135
Name: DataNotificacao, dtype: float64

In [90]:
motivos.sort_values(ascending = False).head(15)

Febre  DificuldadeRespiratoria  Tosse  Coriza  DorGarganta  Diarreia  Cefaleia
Não    Não                      Não    Não     Não          Não       Não         22.047407
Sim    Não                      Sim    Não     Não          Não       Sim          2.927340
                                                                      Não          2.713400
Não    Não                      Sim    Não     Não          Não       Não          2.508824
Sim    Não                      Sim    Sim     Não          Não       Sim          2.465128
Não    Não                      Sim    Sim     Não          Não       Sim          2.385681
                                Não    Não     Não          Não       Sim          2.330352
Sim    Não                      Sim    Sim     Sim          Não       Sim          2.261120
Não    Não                      Sim    Não     Não          Não       Sim          2.211749
Sim    Não                      Não    Não     Não          Não       Sim          2.144219
N

Aproximadamente 21% das pessoas que pertencem a estatística não possuiam sintomas, enquanto 6% possuiam todos.<br>
Analisando os casos positivos:

In [91]:
motivos_conf = df_conf.groupby(sintomas).count().DataNotificacao/df_conf.shape[0]*100
motivos_conf.head()

Febre  DificuldadeRespiratoria  Tosse  Coriza  DorGarganta  Diarreia  Cefaleia
-      -                        -      -       -            -         -           0.873110
                                                                      Não         0.001727
                                                                      Sim         0.000864
                                                            Não       -           0.001727
                                               Não          -         -           0.002591
Name: DataNotificacao, dtype: float64

In [92]:
motivos_conf.sort_values(ascending = False).head(15)

Febre  DificuldadeRespiratoria  Tosse  Coriza  DorGarganta  Diarreia  Cefaleia
Não    Não                      Não    Não     Não          Não       Não         20.709369
Sim    Não                      Sim    Não     Não          Não       Sim          3.859473
                                                                      Não          3.742022
                                       Sim     Não          Não       Sim          2.871503
Não    Não                      Sim    Não     Não          Não       Não          2.588239
Sim    Não                      Não    Não     Não          Não       Sim          2.570967
Não    Não                      Não    Não     Não          Não       Sim          2.342974
                                Sim    Não     Não          Não       Sim          2.307566
Sim    Não                      Sim    Sim     Sim          Não       Sim          2.292885
                                       Não     Sim          Não       Sim          2.176297
N

21% dos casos confirmados não possuiam sintomas, ou seja, eram assintomáticos portadores do COVID, enquanto 3.7% possuiam febre e tosse.<br>
Realizando esta verificação para os casos de óbitos:

In [93]:
df_obitos = df_conf.query("DataObito > 2019")
df_obitos.head()

,DataNotificacao,DataCadastro,DataDiagnostico,DataColeta_RT_PCR,DataColetaTesteRapido,DataEncerramento,DataObito,Classificacao,Evolucao,CriterioConfirmacao,...,ComorbidadeRenal,ComorbidadeDiabetes,ComorbidadeTabagismo,ComorbidadeObesidade,FicouInternado,ViagemBrasil,ViagemInternacional,ProfissionalSaude,PossuiDeficiencia,MoradorDeRua
1070,2020-09-06,2020-09-06,2020-07-15,2020-09-05,2020-09-05,2020-09-07,2020-09-06,Confirmados,Óbito pelo COVID-19,Laboratorial,...,Não,Não,Não,Não,Não Informado,Ignorado,Ignorado,Não,Não,Não
4040,2020-09-04,2020-09-05,2020-08-25,2020-09-04,NaN,2020-09-04,2020-09-04,Confirmados,Óbito pelo COVID-19,Laboratorial,...,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não
5748,2020-09-03,2020-09-03,2020-07-26,2020-08-04,NaN,2020-09-03,2020-08-21,Confirmados,Óbito pelo COVID-19,Laboratorial,...,Não,Não,Não,Não,Ignorado,Não Informado,Não Informado,Não,Não,Não
5779,2020-09-03,2020-09-03,2020-08-05,NaN,2020-08-15,2020-08-31,2020-08-31,Confirmados,Óbito pelo COVID-19,Laboratorial,...,Não,Não,Não,Não,Sim,Ignorado,Ignorado,Não,Não,Não
8241,2020-09-03,2020-09-03,2020-08-30,NaN,2020-09-03,2020-09-04,2020-09-03,Confirmados,Óbito pelo COVID-19,Laboratorial,...,Não,Não,Não,Não,Não Informado,Não Informado,Não Informado,Não,Não,Não


In [94]:
motivos_ob = df_obitos.groupby(sintomas).count().DataNotificacao/df_obitos.shape[0]*100
motivos_ob.head()

Febre  DificuldadeRespiratoria  Tosse  Coriza  DorGarganta  Diarreia  Cefaleia
-      -                        -      -       -            -         -           0.121322
                                                            Não       -           0.030331
                                       Sim     -            -         -           0.030331
       Sim                      -      -       -            -         -           0.151653
                                               Sim          -         -           0.030331
Name: DataNotificacao, dtype: float64

In [95]:
motivos_ob.sort_values(ascending = False).head(10)

Febre  DificuldadeRespiratoria  Tosse  Coriza  DorGarganta  Diarreia  Cefaleia
Sim    Sim                      Sim    Não     Não          Não       Não         11.980588
Não    Sim                      Não    Não     Não          Não       Não          9.311495
       Não                      Não    Não     Não          Não       Não          8.462238
       Sim                      Sim    Não     Não          Não       Não          7.855626
Sim    Não                      Sim    Não     Não          Não       Não          7.127692
       Sim                      Não    Não     Não          Não       Não          6.612072
Não    Não                      Sim    Não     Não          Não       Não          3.579011
Sim    Não                      Não    Não     Não          Não       Não          3.427358
       Sim                      Sim    Não     Não          Não       Sim          3.215044
       Não                      Sim    Não     Não          Não       Sim          2.305126
N

12% possuiam os sintomas clássicos enquanto 8.4% eram assintomáticos.<br>
Com relação as comorbidades, temos o seguinte:

In [96]:
comorbidades = ['ComorbidadePulmao', 'ComorbidadeCardio', 'ComorbidadeRenal', 'ComorbidadeDiabetes', 'ComorbidadeTabagismo', 'ComorbidadeObesidade']
comorb_ob = df_obitos.groupby(comorbidades).count().DataNotificacao/df_obitos.shape[0]*100
comorb_ob.head()

ComorbidadePulmao  ComorbidadeCardio  ComorbidadeRenal  ComorbidadeDiabetes  ComorbidadeTabagismo  ComorbidadeObesidade
-                  -                  -                 -                    -                     -                       0.485290
                                                                                                   Não                     0.030331
                                                                                                   Sim                     0.030331
                                                                             Não                   -                       0.030331
                                                                             Sim                   -                       0.030331
Name: DataNotificacao, dtype: float64

In [97]:
comorb_ob.sort_values(ascending = False).head(10)

ComorbidadePulmao  ComorbidadeCardio  ComorbidadeRenal  ComorbidadeDiabetes  ComorbidadeTabagismo  ComorbidadeObesidade
Não                Não                Não               Não                  Não                   Não                     29.814983
                   Sim                Não               Não                  Não                   Não                     21.443737
                                                        Sim                  Não                   Não                     17.349105
                   Não                Não               Sim                  Não                   Não                      4.367607
                   Sim                Não               Sim                  Não                   Sim                      2.820746
Sim                Sim                Não               Não                  Não                   Não                      2.517440
Não                Sim                Não               Não                  Não  

30% dos mortos não possuiam comorbidades.